In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
mainURL = "https://www.programiz.com/csharp-programming"

page = requests.get(mainURL)
htmlContent = page.content
soup = BeautifulSoup(htmlContent, 'html.parser')

urls = []
pages = soup.find('div', {'class': 'right-bar'}).findAll(href=True)
for slug in pages[:-7]:
    urls.append('https://www.programiz.com' + slug.get('href'))

mainHeadings = []
mainContents = []
videoHeadings = []
videoURLs = []
subHeadingsH2 = []
subHeadingsH3 = []
imageURLs = []
facts = []
definitions = []
explanations = []
codes = []
snippets = []
outputs = []
tableData = []
listItems = []
data = {}


In [3]:
for url in urls:
    page = requests.get(url)
    htmlContent = page.content
    soup = BeautifulSoup(htmlContent, 'html.parser')

    mainHeading = soup.findAll('h1')  # Main Heading
    pageMainHeadings = []
    if mainHeading is not None:
        for head in mainHeading:
            pageMainHeadings.append(head.text)

    mainContent = soup.findAll('p', {"class": "editor-contents__short-description"})  # Main Content
    pageMainContents = []
    if mainContent is not None:
        for mainCon in mainContent:
            pageMainContents.append(mainCon.text)

    videoHeading = soup.findAll('div', {"class": "programiz-youtube-wrapper lite-youtube"})  # Video Heading
    pageVideoHeadings = []
    if videoHeading is not None:
        for vidHead in videoHeading:
            pageVideoHeadings.append(vidHead.find('h2').text)

    videoURL = soup.findAll('div', {"class": "programiz-youtube__container"})  # Video URL
    pageVideoURLs = []
    if videoURL is not None:
        for vid in videoURL:
            pageVideoURLs.append(vid.find('iframe').get('src'))

    subContent = soup.find('div', {"class": "content"})  # Sub Content
    if subContent is None:
        continue

    subHeadingH2 = subContent.findAll('h2')  # Sub Headings h2
    pageSubHeadingsH2 = []
    if subHeadingH2 is not None:
        for head in subHeadingH2:
            pageSubHeadingsH2.append(head.text)

    subHeadingH3 = subContent.findAll('h3')  # Sub Headings h3
    pageSubHeadingsH3 = []
    if subHeadingH3 is not None:
        for head in subHeadingH3:
            pageSubHeadingsH3.append(head.text)

    images = subContent.findAll('img')  # Images
    pageImageURLs = []
    if images is not None:
        for img in images:
            pageImageURLs.append(img.get('src'))

    paragraphs = subContent.findAll('p')  # Paragraphs
    pageFacts = []
    pageDefinitions = []
    pageExplanations = []
    if paragraphs is not None:
        for para in paragraphs:
            slug = para.text
            if 5 <= len(slug) <= 65:
                pageFacts.append(slug)
            elif 66 <= len(slug) <= 126:
                pageDefinitions.append(slug)
            elif 127 < len(slug):
                pageExplanations.append(slug)

    codeContent = subContent.findAll('pre')  # Codes, Snippets and Outputs
    pageCodes = []
    pageSnippets = []
    pageOutputs = []
    for slug in codeContent:
        if slug.code is not None:
            for cod in slug.code.text.split('\n'):
                if cod != '' and not cod.lstrip().rstrip().startswith('//'):
                    if 7 < len(cod) <= 30:
                        pageSnippets.append(cod)
                    elif 31 < len(cod):
                        pageCodes.append(cod)

        else:
            for out in slug.text.split('\n'):
                if out != '' and not out.lstrip().rstrip().startswith('//'):
                    pageOutputs.append(out)
                else:
                    continue

    tables = subContent.findAll('table')  # Tables
    pageTableData = []
    if tables is not None:
        for tb in tables:
            table = ""
            rowData = []
            tableRows = tb.findAll('tr')
            for row in tableRows:
                rowData.append(row.text[1:-1].replace('\n', '|||'))
            for row in rowData:
                table = table + str(row) + '&&&'
            pageTableData.append(table[:-3])

    allListUL = subContent.findAll('ul')  # List Items
    allListOL = subContent.findAll('ol')
    listUL = ""
    listOL = ""
    listItem = ""
    pageListItems = []
    if len(allListUL) != 0 or len(allListOL) != 0:
        if len(allListUL) != 0:
            for lis in allListUL:
                listUL = listUL + str(lis.text).replace('\n', '|||').replace('\t\t', '')
        if len(allListOL) != 0:
            for lis in allListOL:
                listOL = listOL + str(lis.text).replace('\n', '|||').replace('\t\t', '')

        if len(allListUL) != 0 and len(allListOL) == 0:
            listItem = listItem + listUL
        elif len(allListUL) == 0 and len(allListOL) != 0:
            listItem = listItem + listOL
        else:
            listItem = listItem + listUL + '&&&' + listOL
        pageListItems.append(listItem)

    mainHeadings.append(pageMainHeadings)
    mainContents.append(pageMainContents)
    videoHeadings.append(pageVideoHeadings)
    videoURLs.append(pageVideoURLs)
    subHeadingsH2.append(pageSubHeadingsH2)
    subHeadingsH3.append(pageSubHeadingsH3)
    imageURLs.append(pageImageURLs)
    facts.append(pageFacts)
    definitions.append(pageDefinitions)
    explanations.append(pageExplanations)
    snippets.append(pageSnippets)
    codes.append(pageCodes)
    outputs.append(pageOutputs)
    tableData.append(pageTableData)
    listItems.append(pageListItems)


In [4]:
data = {'main_headings': mainHeadings, 'main_contents': mainContents, 'video_headings': videoHeadings,
        'video_urls': videoURLs, 'sub_headings_h2': subHeadingsH2, 'sub_headings_h3': subHeadingsH3,
        'image_urls': imageURLs, 'facts': facts, 'definitions': definitions, 'explanations': explanations,
        'snippets': snippets, 'codes': codes, 'outputs': outputs, 'table_data': tableData, 'list_items': listItems}

df = pd.DataFrame(data)
df.to_csv('Programiz_C#.csv')

In [5]:
a=pd.read_csv("Programiz_C#.csv")
a

,Unnamed: 0,main_headings,main_contents,video_headings,video_urls,sub_headings_h2,sub_headings_h3,image_urls,facts,definitions,explanations,snippets,codes,outputs,table_data,list_items
0,0,['C# Hello World - Your First C# Program'],"['In this tutorial, we will learn how to write...",[],[],"['""Hello World!"" in C#', 'How the ""Hello World...",[],[],"['When you run the program, the output will be...",['The purpose of this program is to get us fam...,['The “Hello World!” program is often the firs...,"['namespace HelloWorld', ' class Hello { ...","[' static void Main(string[] args)', ' ...","['Hello World!', 'System.Console.WriteLine(""He...",[],['Every C# program must have a class definitio...
1,1,['C# Keywords and Identifiers'],"['In this tutorial, we will learn about keywor...",[],[],"['C# Keywords', 'C# Identifiers', 'Rules for N...",['Contextual Keywords'],[],"['For example,', 'The above statement will cre...","['Also, keywords like long, int, char, etc can...",['Keywords are predefined sets of reserved wor...,"['using System;', 'namespace HelloWorld', ' ...","[' static void Main(string[] args)', ' ...","['long mobileNum;', 'long long;', 'int @void;'...",['bstract|||as|||base|||bool&&&reak|||byte|||c...,['An identifier can not be a C# keyword.|||An ...
2,2,['C# Variables and (Primitive) Data Types'],"['In this tutorial, we will learn about variab...",[],[],"['How to declare variables in C#?', 'Rules for...","['Implicitly typed variables', 'Boolean (bool)...",[],"[""Here's an example to declare a variable in C...",['A variable is a symbolic name given to a mem...,['Variables in C# must be declared before they...,"['using System;', 'namespace DataType', ' c...",[' public static void Main(string[] arg...,"['int age;', 'int age;', '... ... ...', 'age =...",['ariable Names|||Remarks&&&ame|||Valid&&&ubje...,['Boolean data type has two possible values: t...
3,3,['C# Operators'],"['In this article, we will learn everything ab...",[],[],"['1. Basic Assignment Operator', '2. Arithmeti...","['Example 1: Basic Assignment Operator', 'Exam...",[],"['Here, 50.05 is assigned to x.', 'When we run...",['Operators are symbols that are used to perfo...,['Operators are used to manipulate variables a...,"['using System;', 'namespace Operator', '\tcla...","['\t\tpublic static void Main(string[] args)',...","['double x;', 'x = 50.05;', 'First Number = 10...",['perator|||Operator Name|||Example&&&|||Addit...,"['If one of the operand is true, the OR operat..."
4,4,['C# Operator Precedence and Associativity'],['In this tutorial we you will learn about ope...,[],[],"['C# Operator Precedence', 'Associativity of O...","['Operator Precedence Table', 'Example 1: Oper...",['//cdn.programiz.com/sites/tutorial2program/f...,"['Take a look at the statement below.', 'What ...","['If addition would have a higher precedence, ...",['Operator precedence is a set of rules which ...,"['using System;', 'namespace Operator', '\tcla...","['\t\tpublic static void Main(string[] args)',...","['int x = 4 + 3 * 5;', '19', 'False', 'result1...",['ategory|||Operators&&&ostfix Increment and D...,"['At first, (--a) is evaluated resulting into ..."
5,5,['C# Bitwise and Bit Shift Operators'],"['In this tutorial, we will learn in detail ab...",[],[],"['Bitwise OR', 'Bitwise AND', 'Bitwise XOR', '...","['Example 1: Bitwise OR', 'Example 2: Bitwise ...",[],"['For Example,', 'Bitwise OR operation between...","['If you are interested to explore more, visit...",['Bitwise and bit shift operators are used to ...,"['using System;', 'namespace Operator', '\tcla...","['\t\tpublic static void Main(string[] args)',...","['14 = 00001110 (In Binary)', '11 = 00001011 (...",['perator|||Operator Name&&&|||Bitwise Complem...,[]
6,6,['C# Basic Input and Output'],"['In this tutorial, we will learn how to take ...",[],[],"['C# Output', 'C# Input']",['Example 1: Printing String using WriteLine()...,[],"['In order to output something in C#, we can u...","['Here, System is a namespace, Console is a cl...",